In [1]:
import requests
import pandas as pd

# Global Request Functions

In [2]:
header = {"X-Riot-Token": "RGAPI-9c25c780-777b-400d-961d-bdfdda5016d7"}


def getSummoner(summoner):
    summoner = requests.get(
        f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner}",
        headers=header,
    )
    return summoner.json()


def getMatchIDs(puuid):
    matchIDs = requests.get(
        f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100",
        headers=header,
    )
    return matchIDs.json()


def getMatch(matchID):
    matches = requests.get(
        f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}",
        headers=header,
    )
    return matches.json()

In [3]:
champions = {}

In [4]:

class Summoner:
    def __init__(self, summonerName):
        self.name = summonerName
        self.puuid = getSummoner(summonerName)["puuid"]
        self.matchIDs = getMatchIDs(self.puuid)
        self.partnerName = ""
        self.partnerPuuid = ""
        self.df = pd.DataFrame(columns=["MatchID", "ChampionID",
                           "PartnerChampionID", "Win/Lost"])

    def initPartner(self, partnerName):
        r = getSummoner(partnerName)
        self.partnerPuuid = r["puuid"]
        self.partnerName = r["name"]

        for i, matchID in enumerate(self.matchIDs):
            matchJSON = getMatch(matchID)
            try:
                participants = matchJSON["info"]["participants"]
            except:
                break

            if self.partnerPuuid in matchJSON["metadata"]["participants"]:
                myNdx = 0
                partnerNdx = 0

                while participants[myNdx]["summonerName"] != self.name:
                    myNdx += 1
                while participants[partnerNdx]["summonerName"] != self.partnerName:
                    partnerNdx += 1

                me = participants[myNdx]
                partner = participants[partnerNdx]

                myChampion = me["championName"]
                partnersChampion = partner["championName"]

                if champions.get(myChampion) == None:
                    champions[myChampion] = me["championId"]
                if champions.get(myChampion) == None:
                    champions[myChampion] = partner["championId"]

                if me["win"]:
                    win = 1
                else:
                    win = 0

                newMatch = pd.Series(
                    [
                        matchID, myChampion, partnersChampion, win
                    ],
                    index=["MatchID", "ChampionID",
                           "PartnerChampionID", "Win/Lost"],
                )

                print(f"Adding: {[matchID, myChampion, partnersChampion, win]} to DF")
                self.df.loc[i] = newMatch

In [5]:
mainSummoner = Summoner("41619")
mainSummoner.initPartner("yongbin")

Adding: ['NA1_4288037065', 'Swain', 'Twitch', 1] to DF
Adding: ['NA1_4288044461', 'Velkoz', 'Azir', 0] to DF
Adding: ['NA1_4288051179', 'Rakan', 'Jinx', 1] to DF
Adding: ['NA1_4287985938', 'Xerath', 'Vayne', 0] to DF
Adding: ['NA1_4287970218', 'Thresh', 'Jhin', 0] to DF
Adding: ['NA1_4287932752', 'Morgana', 'Caitlyn', 0] to DF
Adding: ['NA1_4287868896', 'Urgot', 'Corki', 1] to DF
Adding: ['NA1_4283956369', 'Jhin', 'Vayne', 0] to DF
Adding: ['NA1_4283916232', 'Viego', 'Vayne', 1] to DF
Adding: ['NA1_4283931327', 'Pyke', 'Kaisa', 1] to DF
Adding: ['NA1_4283835083', 'Senna', 'Jhin', 1] to DF
Adding: ['NA1_4283791260', 'Lux', 'Vayne', 1] to DF
Adding: ['NA1_4283674089', 'Bard', 'Ezreal', 1] to DF
Adding: ['NA1_4283557629', 'Velkoz', 'Caitlyn', 1] to DF
Adding: ['NA1_4283519508', 'Nautilus', 'Jhin', 0] to DF
Adding: ['NA1_4281951559', 'Vladimir', 'Veigar', 0] to DF
Adding: ['NA1_4281889703', 'Draven', 'Akali', 0] to DF
Adding: ['NA1_4281916383', 'MasterYi', 'Tristana', 1] to DF
Adding: ['NA

In [6]:
mainSummoner.df.to_csv('41binMatches.csv')


In [7]:
print(champions[67])

KeyError: 67